### Import Libs

In [140]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars xgboost4j-spark-0.72.jar,xgboost4j-0.72.jar pyspark-shell'
import pyspark
from pyspark.sql import SparkSession, functions as F, DataFrame
import pyspark.sql.functions as F
from pyspark.sql.functions import lit
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql import types 
from pyspark.sql.functions import when
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
import pyspark.sql.types as T
from pyspark.sql.functions import desc
spark = SparkSession.builder.getOrCreate()
from pyspark.ml import Pipeline 
from pyspark.ml.feature import * 


spark.sparkContext.addPyFile("sparkxgb.zip")
from sparkxgb import XGBoostEstimator

### Load Data

In [141]:
sprkdf_train = spark.read.csv('train.csv', inferSchema=True, header=True)
sprkdf_test = spark.read.csv('test.csv', inferSchema=True, header=True)
print("Shape of sprkdf_train",(sprkdf_train.count(), len(sprkdf_train.columns)))
print("Shape of sprkdf_test",(sprkdf_test.count(), len(sprkdf_test.columns)))
# sprkdf_train.printSchema()

Shape of sprkdf_train (1460, 81)
Shape of sprkdf_test (1459, 80)


### Categorical data (OneHotEncoder)

In [142]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, VectorAssembler
CATE_FEATURES = ['Alley', 'LotShape', 'LotConfig', 'Neighborhood', 'RoofStyle', 'Foundation', 'Electrical', 'GarageFinish']
stages_pipe = [] # stages in our Pipeline
for categoricalCol in CATE_FEATURES:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()],
                                     outputCols=[categoricalCol + "classVec"])
    stages_pipe += [stringIndexer, encoder]
CATE_FEATURES_name = [x+"classVec" for x in CATE_FEATURES]

### VectorAssembler

In [143]:
numeric_feature = ['TotRmsAbvGrd']

from pyspark.ml.feature import VectorAssembler
VA = VectorAssembler(inputCols=CATE_FEATURES_name + numeric_feature, outputCol="features")
stages_pipe += [VA]

### Training  Model

In [146]:
from pyspark.ml.regression import GBTRegressor
# Train a GBT model.
GBT = GBTRegressor(featuresCol = 'features', labelCol='SalePrice')
stages_pipe += [GBT]

# # XGboost
# XGB = XGBoostEstimator(featuresCol="features", labelCol="SalePrice")
# stages_pipe += [XGB]

### Build and Fit Pipeline

In [149]:
# Build the pipeline 
pipeline = Pipeline(stages=stages_pipe) 

# Fit the pipeline 
model = pipeline.fit(sprkdf_train) 

# Make a prediction 
prediction = model.transform(sprkdf_test)

In [148]:
type(prediction)
df_out = prediction.select('Id',col("prediction").alias("SalePrice"))
# df_out.write.csv('Pyspark_output.csv').save(header = 'true')
df_out.toPandas().to_csv('Pyspark_output.csv', index=False)

### Test XGboost

NameError: name 'XGBoostEstimator' is not defined